In [3]:
import pybedtools
from collections import defaultdict
import pandas as pd

In [2]:
bedtool = pybedtools.BedTool("/projects/ps-yeolab/genomes/mm9/Mus_musculus.NCBIM37.64.fixed.exons.bed")

In [12]:
df = bedtool.to_dataframe()
df['exon_len'] = df.end - df.start

In [43]:
df

,chrom,start,end,name,score,strand,thickStart,thickEnd,exon_len
0,chrY,61650,63038,ENSMUSG00000053211,0,-,0,NaN,1388
1,chrY,63952,64092,ENSMUSG00000053211,0,-,1,NaN,140
2,chrY,66057,66179,ENSMUSG00000053211,0,-,2,NaN,122
3,chrY,69371,69511,ENSMUSG00000053211,0,-,3,NaN,140
4,chrY,71462,71611,ENSMUSG00000053211,0,-,4,NaN,149
5,chrY,75035,75586,ENSMUSG00000053211,0,-,5,NaN,551
6,chrY,96293,96380,ENSMUSG00000053211,0,-,6,NaN,87
7,chrY,133554,133619,ENSMUSG00000053211,0,-,7,NaN,65
8,chrY,133718,133852,ENSMUSG00000053211,0,-,8,NaN,134
9,chrY,155156,155257,ENSMUSG00000069053,0,+,0,NaN,101


In [35]:
result_df = pd.concat([df.groupby("name").first()[['chrom', 'score', 'strand', 'name']],
           df.groupby("name").exon_len.sum(),
           df.groupby("name").start.min(),
           df.groupby("name").end.max(),], axis=1)

result_df['pre_mrna_len'] = result_df.end - result_df.start
result_df['homer_strand'] = result_df.strand.apply(lambda x: 0 if x == "+" else 1)

In [52]:
row

chrom                chr3
score                   0
strand                  -
exon_len             3253
start           107910198
end             107949064
pre_mrna_len        38866
homer_strand            1
Name: ENSMUSG00000000001, dtype: object

In [54]:
with open("/home/gpratt/gscripts/gscripts/clipseq/kasey_peak_calling/clusterid_location_mm9.pre", 'w') as pre:
    with open("/home/gpratt/gscripts/gscripts/clipseq/kasey_peak_calling/clusterid_location_mm9", 'w') as full:
        for name, row in result_df.iterrows():
            pre.write("\t".join(map(str, [row.chrom, row.start, row.end, row.homer_strand, name, row.pre_mrna_len])) + "\n") 
            full.write("\t".join(map(str, [row.chrom, row.start, row.end, row.homer_strand, name, row.exon_len])) + "\n")
